In [3]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
import json
import sqlite3
import sys

In [4]:
ckey=%env API_KEY
csecret=%env API_SECRET_KEY
atoken=%env ACCESS_TOKEN
asecret= %env ACCESS_TOKEN_SECRET

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
api = tweepy.API(auth)

In [88]:
conn = sqlite3.connect('../data/streamingscrap.sqlite')
conn.text_factory = str
cur = conn.cursor()

cur.execute('''DROP TABLE IF EXISTS Tweets ''')
cur.execute('''CREATE TABLE IF NOT EXISTS Tweets
                (id INTEGER PRIMARY KEY, tweet TEXT, username Text, language Text, location Text, tweet_date Text)''')


def store_tweet_stream(item):
    cur.execute('''INSERT OR IGNORE INTO Tweets 
                (tweet, username, language, location, tweet_date) 
                VALUES (?, ?, ?, ?, ?)''',
                (item['tweet'], item['username'], item['language'], item['location'], item['tweet_date']))
    
    conn.commit()
    
def store_tweet_status(tweet):
    cur.execute('''INSERT OR IGNORE INTO Tweets 
                (tweet, username, language, location, tweet_date) 
                VALUES (?, ?, ?, ?, ?)''',
                (tweet, '', '', '', ''))
    
    conn.commit()
    

In [84]:
def to_item(data):
    item = {}
    item['tweet'] = data["text"]
    if 'retweeted_status' in data:
        item['tweet'] = data['retweeted_status']['extended_tweet']["full_text"]
    elif 'extended_tweet' in data:
        item['tweet'] = data['extended_tweet']["full_text"]

    item['username'] = data["user"]["screen_name"]
    item['language'] = data["lang"]
    item['location'] = data["user"]["location"]
    item['tweet_date'] = data['created_at']
    
    return item
    

In [55]:
class listener(StreamListener):

    def on_data(self, data):
        item = to_item(json.loads(data))
        store_tweet(item)
        
        print(f"- {item} \n")

        return True

    def on_error(self, status):
        print(status)
        if status == 420:
            # returning False in on_data disconnects the stream
            return False

In [38]:
twitterStream = Stream(auth, listener(), tweet_mode="extended")

#screenname is not mandatory
streaming_screen_name="MarStebi"
twitterStream.filter(follow=[str(api.get_user(screen_name = streaming_screen_name).id)],
                     track=['test'],
                     is_async=True)



Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mars/.local/share/virtualenvs/Twitter-analysis-3oML7yVh/lib/python3.7/site-packages/tweepy/streaming.py", line 320, in _run
    six.reraise(*exc_info)
  File "/home/mars/.local/share/virtualenvs/Twitter-analysis-3oML7yVh/lib/python3.7/site-packages/six.py", line 693, in reraise
    raise value
  File "/home/mars/.local/share/virtualenvs/Twitter-analysis-3oML7yVh/lib/python3.7/site-packages/tweepy/streaming.py", line 289, in _run
    self._read_loop(resp)
  File "/home/mars/.local/share/virtualenvs/Twitter-analysis-3oML7yVh/lib/python3.7/site-packages/tweepy/streaming.py", line 351, in _read_loop
    self._data(next_status_obj)
  File "/home/mars/.local/share/virtualenvs/Twitter-analysis-3oML7yVh/lib/python3.7/site-packa

In [40]:
#to end streamming
twitterStream.disconnect()


In [78]:
def get_status_tweet(status):
    if hasattr(status, "retweeted_status"):  # Check if Retweet
        try:
            return status.retweeted_status.extended_tweet["full_text"]
        except AttributeError:
            return status.retweeted_status.text
    else:
        try:
            return status.extended_tweet["full_text"]
        except AttributeError:
            return status.text

In [85]:
# by user timeline
for status in tweepy.Cursor(api.user_timeline, id="izzatamar").items(2):
    store_tweet_status(get_status_tweet(status))

In [92]:
# by query
for status in tweepy.Cursor(api.search, q="", lang="EN").items(20):
    print(get_status_tweet(status))

Media Release: Robbery - Male Arrested for Robbery in Niagara Falls  - https://t.co/SApMXRbYqN
@NzStill https://t.co/Cd32actqvJ  Look at at the Snakes of SATAN  55 
55 = same same  insated of gods numer 10 redu… https://t.co/lK4chItuog
@WalkerBragman @mhnsn_ @HoarseWisperer UPDATE: "RICH white male privilege"

https://t.co/UhZli14crh

@Alyssa_Milano @PattyArquette
@WalkerBragman @mhnsn_ @HoarseWisperer UPDATE: "RICH white male privilege"

https://t.co/UhZli14crh

@Alyssa_Milano @PattyArquette
Male involved in suspicious incident involving a child has been identified https://t.co/DBKvjEVxdD

L’individu mêlé… https://t.co/jWLackp4EG
Male involved in suspicious incident involving a child has been identified https://t.co/DBKvjEVxdD

L’individu mêlé… https://t.co/jWLackp4EG


In [ ]:
cur.close()